In [1]:
from dolfin import *
set_log_level(0)

![image](cantilever_beam_fenics_tutorial.png)

In [2]:
#For 2-D sytem
mul = 5
L = 25.
H = 1.
Nx = 250 * mul
Ny = 10 * mul
mesh = RectangleMesh(Point(0., 0.), Point(L, H), Nx, Ny, "crossed")

### Defining the material properties
We will define the properties of the materials in terms of Lame's parameter and type of model. Here, based on the type of model the value of Lame's parameter ($\lambda$ and $\mu$) changes.

* for Plane stress case - $\lambda=\frac{E \nu}{(1+\nu)(1-2 \nu)}$  and  $\quad \mu=\frac{E}{2(1+\nu)}$  
In place stress, the stress is acting in two dimension only, there is no stress in third direction.

* for Plane strain case - $\lambda^{*}=\frac{2 \lambda \mu}{\lambda+2 \mu} $, here $\lambda$ is same as above, but the stress will be defined in terms of $\lambda^{*}$ and $\quad \mu=\frac{E}{2(1+\nu)}$  
In place strain, the systm of stress is such that no strain is produced in third direction, generally the length of third direction(dimension) is considered to be very large.


In [3]:
E = Constant(1e5) #Young's modulus 
nu = Constant(0.3) # Poisson's ratio
model = "plane_stress"

In [4]:
mu = E/2/(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)
if model == "plane_stress":
    lmbda = 2*mu*lmbda/(lmbda+2*mu)

### Defining the function space
As discussed earlier the second module in FEniCS code is to define the Function space.  
Since, here we have a planar system having displacement in two directions so, `VectorFunctionSpace`.

Now,we are defining a function `u_sol` which will store the value of the displacement so, `name` is give as `Displacement` which can be used when visulaizing the data in **Paraview**.


In [5]:
V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)
u_sol = Function(V, name="Displacement")

**Stress** and **Strain** can be defined in terms of Lame's parameter as follows -   
* Stress $\sigma=\lambda \operatorname{tr}(\varepsilon) \mathbf{I}+2 \mu \varepsilon$  

* Strain $\varepsilon=\frac{1}{2}\left((\nabla \mathbf{u})^{T}+\nabla \mathbf{u}\right)$  
[Reference](https://readthedocs.org/projects/comet-fenics/downloads/pdf/latest/)


In [6]:
def sigma(v):
    return lmbda*tr(eps(v))*Identity(2) + 2.0*mu*eps(v)

In [7]:
def eps(v):
    return sym(grad(v))

### Defining the loading function
Now we can define the function to specify the load acting on it, considering here that only dead weight is acting on the body which can be defined in terms of the density of the material.  
`f = Constant((fx,fy))` is used to defined the loading, `fx` and `fy` represent forces in *x* and *y* direction, respectively.

Further, we will ass

In [8]:
rho_g = 1e-3 #Density of the material
f = Constant((0, -rho_g))
f= Constant((0, -rho_g, 0)) # for 3D

### Defining the strong form and Test and Trial functions along with marking the boundary
As discussed earlier, we are using Test and Trial function as `u` and `v` and also defining the strong form of the equation with `E`    
`dot(f,u_sol)` will do the dot(.) product of *f* and *u_sol*  
In FEniCS, we don't need to define the weak form, that part of computation is done by FEniCS itself. `equation = derivative(E, u_sol, v)` and `grad_equation = derivative(equation, u_sol, u)` are used to generate the weak form.

The `CompiledSubDomain` tool in FEniCS is to specify the subdomain in C++ code and thereby speed up our code as FEniCS compile everything in C++ in backend.  
**"on_boundary"** is predefined in FEniCS,for 1-D boundary is point, 2-D it's line and 3-D it's surface.  
x[0], x[1], x[2] = x, y, z.  
`left = CompiledSubDomain("on_boundary && near(x[0], 0, tol)", tol=1e-14)` means we are defining the boundary at $x = 0$ with $tol =10^{-14}$ as "left".

In [9]:
u = TrialFunction(V)
v = TestFunction(V)
E = inner(sigma(u_sol), eps(u_sol))*dx - dot(f,u_sol)*ds()

In [10]:
equation = derivative(E, u_sol, v)
left = CompiledSubDomain("on_boundary && near(x[0], 0, tol)", tol=1e-14)

### Defining the boundary condition
`bc = DirichletBC(V, Constant((0.,0.)), left)` assigns boundary condition to object **bc**, `DirichletBC` is builtin function that takes in three arguments 
* Functionspace `V`
* Value of displacement `Constant((0.,0.))` i.e. displacement in x and y direction is zero
* Location `left` - location of the boundary

### Defining the problem and solver 
`NonlinearVariationalProblem` defines the nature of problem and is assigned to object **problem_disp**  
`NonlinearVariationalSolver` defines the type of solver we want to use which is assigned to object **solver_disp** and takes in **problem_disp** as argument.  
`solver_disp.parameters` defines the parameters for the solver which is assigned to **prm_disp**, that can be used to define the type of solver and it's parameters.

In [11]:
bc = DirichletBC(V, Constant((0.,0.)), left)

grad_equation = derivative(equation, u_sol, u)

problem_disp = NonlinearVariationalProblem(
                    equation, u_sol, bc, grad_equation)

solver_disp = NonlinearVariationalSolver(problem_disp)
prm_disp = solver_disp.parameters

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


### Defining the solver type along with it's parameters and preconditioning
There are various solvers and preconditionaers available in FEniCS, based on the nature of the problem we select a particular solvers. Every solver has some default parameters,which can be modified by using following command-  
`prm_disp["Solver_type"]["Parameter"] = value`
The list of various solvers and preconditioners can be seen in figure.
![figure](FEniCS@2x.pn)

**Reports** are used when there is some bug, and as the name suggest report will store some data which occupies alot of space so generally we keep it off, untill there is need for debugging. 

Sparse LU decomposition (Gaussian elimination) is used by default to solve linear systems of equations in FEniCS programs.However, sparse LU decomposition becomes slow and one quickly runs out of memory for larger problems. For large problems, we instead need to use iterative methods which are faster and require much less memory. 
As per my knowledge, preconditioners are used to condition or modify the irregular generated so as to ease the computing and enhance the performance.

In [12]:
solvers = (
    "bicgstab",
    "cg",
    "default",
    "gmres",
    "minres",
    "mumps",
    "petsc",
    "richardson",
    "superlu",
    "tfqmr",
    "umfpack",
)
preconditioners = (
    "amg",
    "default",
    "hypre_amg",
    "hypre_euclid",
    "hypre_parasails",
    "icc",
    "ilu",
    "jacobi",
    "none",
    "petsc_amg",
    "sor",
)
linesearch = ("basic", "bt", "cp", "l2", "nleqerr")
prm_disp["nonlinear_solver"] = "newton"
prm_disp["newton_solver"]["maximum_iterations"] = 100000
prm_disp["newton_solver"]["report"] = False    
prm_disp["newton_solver"]["absolute_tolerance"] = 1e-5
prm_disp["newton_solver"]["relative_tolerance"] = 1e-7
prm_disp["newton_solver"]["linear_solver"] = "cg"
prm_disp["newton_solver"]["preconditioner"] = "default"

#prm_disp["newton_solver"]["lu_solver"]["report"] = True
prm_disp["newton_solver"]["krylov_solver"]["report"] = True
#prm_disp["newton_solver"]["krylov_solver"]["error_on_nonconvergence"] = True
#prm_disp["newton_solver"]["krylov_solver"]["absolute_tolerance"] = 1e-7
#prm_disp["newton_solver"]["krylov_solver"]["relative_tolerance"] = 1e-5
#prm_disp["newton_solver"]["krylov_solver"]["maximum_iterations"] = 100
#prm_disp["newton_solver"]["krylov_solver"]["nonzero_initial_guess"] = True

### Printing the result 
`print("Maximal deflection:", -u_sol(L,H/2.)[1])` prints output at particular point **(L,H/2)** and **x[1]** tells that **y** coordinate is printed at that point.

In [13]:
solver_disp.solve()



print("Maximal deflection:", -u_sol(L,H/2.)[1])

Maximal deflection: 0.006004162178146888
